# Работа с текстом и бустинг

В этом блоке мы попробуем поработать с текстовым признаком notes, который содержит сводки новостей о событиях. Для его обработки мы будем использовать tensorflow, и обучать кэтбуст

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [34]:
df = pd.read_csv('df_vib.csv')

In [36]:
df_boosting = df[['NOTES','index', 'classification']]

In [ ]:
!pip install -U tensorflow
!pip install -U tensorflow-text
import tensorflow as tf
import tensorflow_text as text



Вот наша функция для векторизации текстовыз данных

In [37]:
import tensorflow_hub as hub
import numpy as np

encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

def vectorize_sentence(sentence):
    embeddings = encoder([sentence])
    return embeddings.numpy()

sentence_vectors = []

for sentence in df_boosting['NOTES']:
    vector = vectorize_sentence(sentence)
    sentence_vectors.append(vector)

sentence_vectors = np.vstack(sentence_vectors)

In [8]:
sentence_vectors.shape

(10000, 512)

Вот мы и получили из наших заметок целых 512 новых признаков

In [10]:
# закодируем наш таргет
tf = {"High income": 4, "Upper middle income": 3, 'Lower middle income':2, 'Low income':1}
df["classification"] = df["classification"].replace(tf)
target = df["classification"]

**Классификация по тексту**

In [ ]:
!pip install catboost

In [12]:
#делаем сплит
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(sentence_vectors, target,  test_size = 0.2, random_state = 322)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

model = CatBoostClassifier(
    iterations=100,
    random_seed=322)

model.fit(x_train, y_train)



In [15]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)
print(f'f1: {f1_score(y_test, y_pred, average = "macro")} ' )
print(f'roc_auc: {roc_auc_score(y_test, y_pred_proba, multi_class= "ovo")}')


f1: 0.7194434061186197 
roc_auc: 0.9142858048061768


Удивительно, но нам удалось побить все метрики классификации, которые мы получили по остальным признакам, одним только текстом и бустингом, roc_auc достиг целых 0.91

**Регрессия по тексту**

In [16]:
from catboost import CatBoostRegressor

In [38]:
# Так же делаем целевой индекс человеческого развития от 0 до 100, чтоб mse было адекватным
target1 = df_boosting['index']*100

In [39]:
# делаем сплит
x_train1, x_test1, y_train1, y_test1 = train_test_split(sentence_vectors, target1,  test_size = 0.2, random_state = 42)

In [ ]:
model1 = CatBoostRegressor(
    iterations=100,
    random_seed=42)
model1.fit(x_train1, y_train1)

In [42]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
y_pred = model1.predict(x_test1)
print(f'MSE: {mse(y_test1, y_pred)}')
print(f'MAE: {mae(y_test1, y_pred)}')

MSE: 93.40674245187489
MAE: 7.409860725892147


Как ни странно, обучение на текстовом признаке показало лучшие метрики и для задачи регрессии

**Теперь уже точно можно сказать, что зависимость между уровнем насилия и развитием страны есть, ведь и для задачи классификации, и для задачи регрессии мы получили хорошие метрики, более того, именно текстовый анализ показал лучший результат. Это, в свою очередь, значит, что построенные нами модели обладают хорошей предсказательной силой для прогнозирования индекса развития и класса страны на основе уровня насилия.**